In [4]:
from selenium.webdriver.common.keys import Keys
# import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
# from lxml import etree
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
url = "https://lims.minneapolismn.gov/MarkedAgenda/3323"
driver.get(url)

C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_8644\514903665.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


In [7]:
# Find all minutes_items urls
driver.find_element(by='css selector', value='#fileGroup_0 .custom-rcaview-action-review a').get_attribute('href')
hrefs = driver.find_elements(by='css selector', value='.custom-rcaview-action-review a')
hrefs = [i.get_attribute('href') for i in hrefs]

In [57]:
li = driver.find_elements(by='css selector', value='li.customLineHeight')
li[1].find_element(by='css selector', value='a.hrefnowrap').get_attribute('href')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a.hrefnowrap"}
  (Session info: chrome=103.0.5060.134)
Stacktrace:
Backtrace:
	Ordinal0 [0x00505FD3+2187219]
	Ordinal0 [0x0049E6D1+1763025]
	Ordinal0 [0x003B3E78+802424]
	Ordinal0 [0x003E1C10+990224]
	Ordinal0 [0x003E1EAB+990891]
	Ordinal0 [0x003D8411+951313]
	Ordinal0 [0x003FCBD4+1100756]
	Ordinal0 [0x003D8394+951188]
	Ordinal0 [0x003FCDA4+1101220]
	Ordinal0 [0x0040CFC2+1167298]
	Ordinal0 [0x003FC9A6+1100198]
	Ordinal0 [0x003D6F80+946048]
	Ordinal0 [0x003D7E76+949878]
	GetHandleVerifier [0x007A90C2+2721218]
	GetHandleVerifier [0x0079AAF0+2662384]
	GetHandleVerifier [0x0059137A+526458]
	GetHandleVerifier [0x00590416+522518]
	Ordinal0 [0x004A4EAB+1789611]
	Ordinal0 [0x004A97A8+1808296]
	Ordinal0 [0x004A9895+1808533]
	Ordinal0 [0x004B26C1+1844929]
	BaseThreadInitThunk [0x76DEFA29+25]
	RtlGetAppContainerNamedObjectPath [0x774C7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x774C

In [58]:
driver.quit()